# Compare Nutrition in Fast Food Items

In [9]:
# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress, ttest_ind
import scipy.stats as stats
from scipy import stats
import pygwalker as pyg
import sys
import requests
from ipywidgets import interact, widgets, fixed
from IPython.display import display
import matplotlib.patheffects as path_effects
import altair as alt


In [10]:
# Load CSVs
menu_df = pd.read_csv('Data/fastfood.csv')
menu_df.head()

,restaurant,item,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein,vit_a,vit_c,calcium,salad
0,Mcdonalds,Artisan Grilled Chicken Sandwich,380,60,7,2.0,0.0,95,1110,44,3.0,11,37.0,4.0,20.0,20.0,Other
1,Mcdonalds,Single Bacon Smokehouse Burger,840,410,45,17.0,1.5,130,1580,62,2.0,18,46.0,6.0,20.0,20.0,Other
2,Mcdonalds,Double Bacon Smokehouse Burger,1130,600,67,27.0,3.0,220,1920,63,3.0,18,70.0,10.0,20.0,50.0,Other
3,Mcdonalds,Grilled Bacon Smokehouse Chicken Sandwich,750,280,31,10.0,0.5,155,1940,62,2.0,18,55.0,6.0,25.0,20.0,Other
4,Mcdonalds,Crispy Bacon Smokehouse Chicken Sandwich,920,410,45,12.0,0.5,120,1980,81,4.0,18,46.0,6.0,20.0,20.0,Other


In [11]:
# Get information on df
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   restaurant   515 non-null    object 
 1   item         515 non-null    object 
 2   calories     515 non-null    int64  
 3   cal_fat      515 non-null    int64  
 4   total_fat    515 non-null    int64  
 5   sat_fat      515 non-null    float64
 6   trans_fat    515 non-null    float64
 7   cholesterol  515 non-null    int64  
 8   sodium       515 non-null    int64  
 9   total_carb   515 non-null    int64  
 10  fiber        503 non-null    float64
 11  sugar        515 non-null    int64  
 12  protein      514 non-null    float64
 13  vit_a        301 non-null    float64
 14  vit_c        305 non-null    float64
 15  calcium      305 non-null    float64
 16  salad        515 non-null    object 
dtypes: float64(7), int64(7), object(3)
memory usage: 68.5+ KB


In [12]:
# Drop Columns missing several values
# Dropping Salad because it's not a useful field
menu_df = menu_df.drop(labels=['vit_a', 'vit_c', 'calcium', 'salad'], axis=1)

# Check to ensure it worked
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   restaurant   515 non-null    object 
 1   item         515 non-null    object 
 2   calories     515 non-null    int64  
 3   cal_fat      515 non-null    int64  
 4   total_fat    515 non-null    int64  
 5   sat_fat      515 non-null    float64
 6   trans_fat    515 non-null    float64
 7   cholesterol  515 non-null    int64  
 8   sodium       515 non-null    int64  
 9   total_carb   515 non-null    int64  
 10  fiber        503 non-null    float64
 11  sugar        515 non-null    int64  
 12  protein      514 non-null    float64
dtypes: float64(4), int64(7), object(2)
memory usage: 52.4+ KB


In [13]:
# Clean Fiber and Protien missing values
# Calculate column averages
fiber_avg = menu_df['fiber'].mean()
protein_avg = menu_df['protein'].mean()

# Fill missing values with averages
menu_df['fiber'].fillna(fiber_avg, inplace=True)
menu_df['protein'].fillna(protein_avg, inplace=True)

# Verify the changes
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   restaurant   515 non-null    object 
 1   item         515 non-null    object 
 2   calories     515 non-null    int64  
 3   cal_fat      515 non-null    int64  
 4   total_fat    515 non-null    int64  
 5   sat_fat      515 non-null    float64
 6   trans_fat    515 non-null    float64
 7   cholesterol  515 non-null    int64  
 8   sodium       515 non-null    int64  
 9   total_carb   515 non-null    int64  
 10  fiber        515 non-null    float64
 11  sugar        515 non-null    int64  
 12  protein      515 non-null    float64
dtypes: float64(4), int64(7), object(2)
memory usage: 52.4+ KB


In [14]:
# Convert all integer columns to floats to make it easier to compare
int_columns = menu_df.select_dtypes(include='int64').columns
menu_df[int_columns] = menu_df[int_columns].astype(float)

# Create a new column to display the item with the chain name after, making it easier to read the drop down menus
menu_df.insert(2, 'item_with_chain', menu_df['item'] + ' (' + menu_df['restaurant'] + ')')

# Verify the changes
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   restaurant       515 non-null    object 
 1   item             515 non-null    object 
 2   item_with_chain  515 non-null    object 
 3   calories         515 non-null    float64
 4   cal_fat          515 non-null    float64
 5   total_fat        515 non-null    float64
 6   sat_fat          515 non-null    float64
 7   trans_fat        515 non-null    float64
 8   cholesterol      515 non-null    float64
 9   sodium           515 non-null    float64
 10  total_carb       515 non-null    float64
 11  fiber            515 non-null    float64
 12  sugar            515 non-null    float64
 13  protein          515 non-null    float64
dtypes: float64(11), object(3)
memory usage: 56.5+ KB


In [15]:
menu_df.head(3)

,restaurant,item,item_with_chain,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein
0,Mcdonalds,Artisan Grilled Chicken Sandwich,Artisan Grilled Chicken Sandwich (Mcdonalds),380.0,60.0,7.0,2.0,0.0,95.0,1110.0,44.0,3.0,11.0,37.0
1,Mcdonalds,Single Bacon Smokehouse Burger,Single Bacon Smokehouse Burger (Mcdonalds),840.0,410.0,45.0,17.0,1.5,130.0,1580.0,62.0,2.0,18.0,46.0
2,Mcdonalds,Double Bacon Smokehouse Burger,Double Bacon Smokehouse Burger (Mcdonalds),1130.0,600.0,67.0,27.0,3.0,220.0,1920.0,63.0,3.0,18.0,70.0


In [16]:
# Find duplicate items in 'item_with_chain'
print(menu_df['item_with_chain'].value_counts())

duplicate_items = menu_df[menu_df.duplicated(subset='item_with_chain', keep=False)]

# Display duplicate items
print(duplicate_items[['item_with_chain', 'calories', 'cal_fat', 'sat_fat', \
                       'trans_fat', 'cholesterol', 'sodium', 'total_carb', 'fiber', 'sugar', 'protein']])

# Remove one of the duplicates (you can choose 'first' or 'last')
menu_df = menu_df.drop_duplicates(subset='item_with_chain', keep='first')

Express Taco Salad w/ Chips (Taco Bell)         2
Chili Cheese Burrito (Taco Bell)                2
Artisan Grilled Chicken Sandwich (Mcdonalds)    1
6" Sweet Onion Chicken Teriyaki (Subway)        1
Footlong Subway Seafood Sensation (Subway)      1
                                               ..
Roast Turkey & Swiss Sandwich (Arbys)           1
Roast Beef Gyro (Arbys)                         1
Reuben Sandwich (Arbys)                         1
5 piece Prime-Cut Chicken Tenders (Arbys)       1
Fiesta Taco Salad-Steak (Taco Bell)             1
Name: item_with_chain, Length: 513, dtype: int64
                             item_with_chain  calories  cal_fat  sat_fat  \
414         Chili Cheese Burrito (Taco Bell)     380.0    150.0      8.0   
492         Chili Cheese Burrito (Taco Bell)     380.0    150.0      8.0   
497  Express Taco Salad w/ Chips (Taco Bell)     580.0    260.0      9.0   
511  Express Taco Salad w/ Chips (Taco Bell)     580.0    260.0      9.0   

     trans_fat  chol

In [17]:
duplicate_items2 = menu_df[menu_df.duplicated(subset='item_with_chain', keep=False)]
print(duplicate_items2)

Empty DataFrame
Columns: [restaurant, item, item_with_chain, calories, cal_fat, total_fat, sat_fat, trans_fat, cholesterol, sodium, total_carb, fiber, sugar, protein]
Index: []


In [18]:
# Look at columns
menu_df.columns

Index(['restaurant', 'item', 'item_with_chain', 'calories', 'cal_fat',
       'total_fat', 'sat_fat', 'trans_fat', 'cholesterol', 'sodium',
       'total_carb', 'fiber', 'sugar', 'protein'],
      dtype='object')

In [19]:
# Look at restaurants 
menu_df.restaurant.unique()

array(['Mcdonalds', 'Chick Fil-A', 'Sonic', 'Arbys', 'Burger King',
       'Dairy Queen', 'Subway', 'Taco Bell'], dtype=object)

In [20]:
# Look at Menu Items
menu_df.item.unique()

array(['Artisan Grilled Chicken Sandwich',
       'Single Bacon Smokehouse Burger', 'Double Bacon Smokehouse Burger',
       'Grilled Bacon Smokehouse Chicken Sandwich',
       'Crispy Bacon Smokehouse Chicken Sandwich', 'Big Mac',
       'Cheeseburger', 'Classic Chicken Sandwich', 'Double Cheeseburger',
       'Double Quarter Pounder® with Cheese', 'Filet-O-Fish®',
       'Garlic White Cheddar Burger',
       'Grilled Garlic White Cheddar Chicken Sandwich',
       'Crispy Garlic White Cheddar Chicken Sandwich', 'Hamburger',
       'Lobster Roll', 'Maple Bacon Dijon 1/4 lb Burger',
       'Grilled Maple Bacon Dijon Chicken Sandwich',
       'Crispy Maple Bacon Dijon Chicken Sandwich', 'McChicken',
       'McDouble', 'McRib', 'Pico Guacamole 1/4 lb Burger',
       'Grilled Pico Guacamole Chicken Sandwich',
       'Crispy Pico Guacamole Chicken Sandwich',
       'Premium Buttermilk Crispy Chicken Deluxe Sandwich',
       'Premium Crispy Chicken Deluxe Sandwich',
       'Quarter Pounder® 

In [21]:
# Define a function to display information for the selected item
def display_item_info(selected_item):
    item_data = menu_df[menu_df['item_with_chain'] == selected_item].squeeze()
    display(item_data)

# Create a dropdown widget with the list of items
item_dropdown = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item:')

# Connect the dropdown widget to the display_item_info function
interact(display_item_info, selected_item=item_dropdown)

interactive(children=(Dropdown(description='Select Item:', options=('Artisan Grilled Chicken Sandwich (Mcdonal…

<function __main__.display_item_info(selected_item)>

In [22]:
# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    display(pd.concat([item_data1, item_data2], axis=1, keys=['Item 1', 'Item 2']))

# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2)>

# Graphs

In [23]:

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2, selected_nutrient):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=['Item 1', 'Item 2'])
    
    plt.figure(figsize=(10, 6))
    comparison_df.loc[selected_nutrient].plot(kind='bar', color=['skyblue', 'lightcoral'])
    plt.title(f"{selected_nutrient} Comparison between {selected_item1} and {selected_item2}")
    plt.xlabel(selected_nutrient)
    plt.ylabel("Values")
    plt.show()

# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:')

# Create a dropdown widget with the list of nutrients
nutrient_dropdown = widgets.Dropdown(options=menu_df.columns[3:], description='Select Nutrient:')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2, selected_nutrient=nutrient_dropdown)

interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2, selected_nutrient)>

In [24]:
# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2, selected_nutrient):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=['Item 1', 'Item 2'])
    
    plt.figure(figsize=(10, 6))
    ax = comparison_df.loc[selected_nutrient].plot(kind='bar', color=['skyblue', 'lightcoral'])
    
    plt.title(f"{selected_nutrient} Comparison between {selected_item1} and {selected_item2}")
    plt.xlabel(selected_nutrient)
    plt.ylabel("Values")

    # Add labels on top of the bars
    for i, v in enumerate(comparison_df.loc[selected_nutrient]):
        ax.text(i, v + 0.1, str(round(v, 2)), ha='center', va='bottom')

    plt.show()

# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:')

# Create a dropdown widget with the list of nutrients
nutrient_dropdown = widgets.Dropdown(options=menu_df.columns[3:], description='Select Nutrient:')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2, selected_nutrient=nutrient_dropdown)

interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2, selected_nutrient)>

In [25]:
!pip install -U altair_viewer


In [26]:
# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2, selected_nutrient):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=['Item 1', 'Item 2'])
    
    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name=selected_nutrient)

    chart = alt.Chart(melted_df).mark_bar().encode(
        x='index:N',
        y=alt.Y(f'{selected_nutrient}:Q', title='Values'),
        color='Item:N',
        tooltip=['Item:N', alt.Tooltip(f'{selected_nutrient}:Q', title=selected_nutrient)],
    ).properties(
        title=f"{selected_nutrient} Comparison between {selected_item1} and {selected_item2}"
    )

    chart.show()

# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:')

# Create a dropdown widget with the list of nutrients
nutrient_dropdown = widgets.Dropdown(options=menu_df.columns[3:], description='Select Nutrient:')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2, selected_nutrient=nutrient_dropdown)

interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2, selected_nutrient)>

In [27]:
# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2, selected_nutrient):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=['Item 1', 'Item 2'])
    
    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name=selected_nutrient)

    chart = alt.Chart(melted_df).mark_bar().encode(
        x='index:N',
        y=alt.Y(f'{selected_nutrient}:Q', title='Values'),
        color='Item:N',
        tooltip=['Item:N', alt.Tooltip(f'{selected_nutrient}:Q', title=selected_nutrient)],
    ).properties(
        title=f"{selected_nutrient} Comparison between {selected_item1} and {selected_item2}"
    )

    chart.display()

# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:')

# Create a dropdown widget with the list of nutrients
nutrient_dropdown = widgets.Dropdown(options=menu_df.columns[3:], description='Select Nutrient:')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2, selected_nutrient=nutrient_dropdown)

interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2, selected_nutrient)>

In [28]:
# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2, selected_nutrient):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=['Item 1', 'Item 2'])
    
    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name=selected_nutrient)

    # Adjust the chart to display individual bars with transparency
    chart = alt.Chart(melted_df).mark_bar().encode(
        x='index:N',
        y=alt.Y(f'{selected_nutrient}:Q', title='Values'),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#1f77b4', '#ff7f0e']), legend=None),
        opacity=alt.condition(alt.datum.Item == 'Item 2', alt.value(0.7), alt.value(1.0)),  # Adjust transparency
        tooltip=['Item:N', alt.Tooltip(f'{selected_nutrient}:Q', title=selected_nutrient)],
    ).properties(
        title=f"{selected_nutrient} Comparison between {selected_item1} and {selected_item2}"
    )

    chart.display()


# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:')

# Create a widget to input the selected nutrient directly
nutrient_input = widgets.Text(value='Protein', description='Selected Nutrient:')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2, selected_nutrient=fixed('Protein'))


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2, selected_nutrient)>

In [29]:
import pandas as pd
import altair as alt
from IPython.display import display
from ipywidgets import interact, widgets, fixed

# Assuming you have the DataFrame named menu_df

# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2, selected_nutrient):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=['Item 1', 'Item 2'])
    
    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name=selected_nutrient)

    # Adjust the chart to display individual bars side by side
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title='Nutrient'),
        y=alt.Y(f'{selected_nutrient}:Q', title='Values'),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#1f77b4', '#ff7f0e']), legend=None),
        column='Item:N',
        tooltip=['Item:N', alt.Tooltip(f'{selected_nutrient}:Q', title=selected_nutrient)],
    ).properties(
        width=300,
        title=f"{selected_nutrient} Comparison between {selected_item1} and {selected_item2}"
    )

    display(chart)


# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:')

# Create a widget to input the selected nutrient directly
nutrient_input = widgets.Text(value='Protein', description='Selected Nutrient:')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2, selected_nutrient=nutrient_input)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2, selected_nutrient)>

Trying to fix this one

In [30]:
import pandas as pd
import altair as alt
from IPython.display import display
from ipywidgets import interact, widgets, fixed

# Assuming you have the DataFrame named menu_df

# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=['Item 1', 'Item 2'])
    
    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name='Value')

    # Adjust the chart to display individual bars side by side
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title='Nutrient'),
        y=alt.Y('Value:Q', title='Values'),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#1f77b4', '#ff7f0e']), legend=None),
        column='Item:N',
        tooltip=['Item:N', alt.Tooltip('Value:Q', title='Value')],
    ).properties(
        width=300,
        title=f"Nutrient Comparison between {selected_item1} and {selected_item2}"
    )

    display(chart)


# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2)>

In [31]:
import pandas as pd
import altair as alt
from IPython.display import display
from ipywidgets import interact, widgets, fixed

# Assuming you have the DataFrame named menu_df

# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=['Item 1', 'Item 2'])
    
    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name='Value')

    # Adjust the chart to display individual bars side by side
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title='Nutrient'),
        y=alt.Y('Value:Q', title='Values'),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#1f77b4', '#ff7f0e']), legend=None),
        tooltip=['Item:N', alt.Tooltip('Value:Q', title='Value')],
    ).properties(
        width=300,
        title="Comparing Nutrition"
    ).facet(
        row='Item:N',
        title="Comparing Nutrition"
    )

    display(chart)


# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2)>

In [32]:
import pandas as pd
import altair as alt
from IPython.display import display
from ipywidgets import interact, widgets, fixed

# Assuming you have the DataFrame named menu_df

# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=['Item 1', 'Item 2'])
    
    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name='Value')

    # Adjust the chart to display individual bars side by side
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title='Nutrient'),
        y=alt.Y('Value:Q', title='Values'),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#1f77b4', '#ff7f0e']), legend=None),
        tooltip=['Item:N', alt.Tooltip('Value:Q', title='Value')],
    ).properties(
        width=300,
        title="Comparing Nutrition"
    ).facet(
        column='Item:N',
        title="Item 2"
    )

    display(chart)


# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2)>

In [33]:
import pandas as pd
import altair as alt
from IPython.display import display
from ipywidgets import interact, widgets, fixed

# Assuming you have the DataFrame named menu_df

# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=['Item 1', 'Item 2'])
    
    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name='Value')

    # Adjust the chart to display individual bars side by side
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title='Nutrient'),
        y=alt.Y('Value:Q', title='Values'),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#1f77b4', '#ff7f0e']), legend=None),
        tooltip=['Item:N', alt.Tooltip('Value:Q', title='Value')],
    ).properties(
        width=300,
        title="Comparing Nutrition"
    ).facet(
        column='Item:N',
        title='Item'  # Set a static title for each chart
    )

    display(chart)


# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2)>

In [34]:
import pandas as pd
import altair as alt
from IPython.display import display
from ipywidgets import interact, widgets, fixed

# Assuming you have the DataFrame named menu_df

# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=['Item 1', 'Item 2'])
    
    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name='Value')

    # Adjust the chart to display individual bars side by side
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title='Nutrient'),
        y=alt.Y('Value:Q', title='Values'),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#1f77b4', '#ff7f0e']), legend=None),
        tooltip=['Item:N', alt.Tooltip('Value:Q', title='Value')],
    ).properties(
        width=300,
        title="Comparing Nutrition"
    ).facet(
        column='Item:N',
        title=alt.TitleParams(text=selected_item1, offset=10, orient='top')  # Set dynamic title for each chart
    )

    display(chart)


# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2)>

Attempting to change chart titles:

In [35]:
import pandas as pd
import altair as alt
from IPython.display import display
from ipywidgets import interact, widgets, fixed

# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=['Item 1', 'Item 2'])

    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name='Value')

    # Adjust the chart to display individual bars side by side
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title=' '),
        y=alt.Y('Value:Q', title=' '),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#1f77b4', '#ff7f0e']), legend=None),
        tooltip=['Item:N', alt.Tooltip('Value:Q', title='Value')],
    ).properties(
        width=300,
        title=" "
    ).facet(
        column='Item:N',
        title=alt.TitleParams(text=selected_item1, offset=10, orient='top')  # Set dynamic title for each chart
    )

    display(chart)


# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2)>

In [36]:
import pandas as pd
import altair as alt
from IPython.display import display
from ipywidgets import interact, widgets, fixed

# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    # Update the keys with the string values of the selected items
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=[selected_item1, selected_item2])

    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name='Value')

    # Adjust the chart to display individual bars side by side
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title=' '),
        y=alt.Y('Value:Q', title=' '),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#1f77b4', '#ff7f0e']), legend=None),
        tooltip=['Item:N', alt.Tooltip('Value:Q', title='Value')],
    ).properties(
        width=300,
        title=" "
    ).facet(
        column='Item:N',
        title=alt.TitleParams(text="Compare Nutrients", offset=10, orient='top')  # Set dynamic title for each chart
    )

    display(chart)


# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2)>

## This is a good one

In [37]:
import pandas as pd
import altair as alt
from IPython.display import display
from ipywidgets import interact, widgets, fixed

# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    # Update the keys with the string values of the selected items
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=[selected_item1, selected_item2])

    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name='Value')

    # Adjust the chart to display individual bars side by side
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title=' '),
        y=alt.Y('Value:Q', title=' '),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#6495ED', '#ADD8E6']), legend=None),
        tooltip=['Item:N', alt.Tooltip('Value:Q', title='Value')],
    ).properties(
        width=300,
        title=" "
    ).facet(
        column='Item:N',
        title=alt.TitleParams(text="Comparing the Nutrients of the Two Selected Items", offset=10, orient='top')
    )

    display(chart)



# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:', value=menu_df['item_with_chain'].unique()[1])

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2)>

In [38]:
import pandas as pd
import altair as alt
from IPython.display import display
from ipywidgets import interact, widgets, fixed

# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()
    
    # Filter only the 'Calories' column for both items
    item_data1 = item_data1[['calories']]
    item_data2 = item_data2[['calories']]

    # Update the keys with the string values of the selected items
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=[selected_item1, selected_item2])

    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name='Value')

    # Adjust the chart to display individual bars side by side
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title=' '),
        y=alt.Y('Value:Q', title=' '),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#6495ED', '#ADD8E6']), legend=None),
        tooltip=['Item:N', alt.Tooltip('Value:Q', title='Value')],
    ).properties(
        width=300,
        title=" "
    ).facet(
        column='Item:N',
        title=alt.TitleParams(text="Comparing the Calories of the Two Selected Items", offset=10, orient='top')
    )

    display(chart)

# Create two dropdown widgets with the list of items
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:', value=menu_df['item_with_chain'].unique()[1])

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2)>

In [39]:
import pandas as pd
import altair as alt
from IPython.display import display
from ipywidgets import interact, widgets, fixed

# Enable notebook rendering for Altair
alt.renderers.enable('default')

# Define a function to display information for the selected items
def display_items_info(selected_item1, selected_item2, selected_column):
    item_data1 = menu_df[menu_df['item_with_chain'] == selected_item1].squeeze()
    item_data2 = menu_df[menu_df['item_with_chain'] == selected_item2].squeeze()

    # Filter only the selected column for both items
    item_data1 = item_data1[[selected_column]]
    item_data2 = item_data2[[selected_column]]

    # Update the keys with the string values of the selected items
    comparison_df = pd.concat([item_data1, item_data2], axis=1, keys=[selected_item1, selected_item2])

    # Melt the DataFrame for Altair
    melted_df = comparison_df.reset_index().melt(id_vars=['index'], var_name='Item', value_name='Value')

    # Adjust the chart to display individual bars side by side
    chart = alt.Chart(melted_df).mark_bar().encode(
        x=alt.X('index:N', title=' '),
        y=alt.Y('Value:Q', title=' '),
        color=alt.Color('Item:N', scale=alt.Scale(range=['#6495ED', '#ADD8E6']), legend=None),
        tooltip=['Item:N', alt.Tooltip('Value:Q', title='Value')],
    ).properties(
        width=300,
        title=" "
    ).facet(
        column='Item:N',
        title=alt.TitleParams(text=f"Comparing {selected_column} of the Two Selected Items", offset=10, orient='top')
    )

    display(chart)

# Create dropdown widgets for items and columns
item_dropdown1 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 1:')
item_dropdown2 = widgets.Dropdown(options=menu_df['item_with_chain'].unique(), description='Select Item 2:', value=menu_df['item_with_chain'].unique()[1])
column_dropdown = widgets.Dropdown(options=menu_df.columns[3:].tolist(), description='Select Column:', value='calories')

# Connect the dropdown widgets to the display_items_info function
interact(display_items_info, selected_item1=item_dropdown1, selected_item2=item_dropdown2, selected_column=column_dropdown)


interactive(children=(Dropdown(description='Select Item 1:', options=('Artisan Grilled Chicken Sandwich (Mcdon…

<function __main__.display_items_info(selected_item1, selected_item2, selected_column)>